# This file is for gathering all the data from government websites for use later

In [1]:
from bs4 import BeautifulSoup
import requests, json
import pandas as pd
import numpy as np

### Set some global variables

In [2]:
with open('api_keys.txt') as file:
    api = json.loads(file.read())['api']
since = 2000
to = 2018
column_names_xls = ['State FIPS Code', 'Postal Code', 'Name', 'Poverty Estimate, All Ages',
       '90% CI Lower Bound', '90% CI Upper Bound', 'Poverty Percent, All Ages',
       '90% CI Lower Bound', '90% CI Upper Bound',
       'Poverty Estimate, Age 0-17', '90% CI Lower Bound',
       '90% CI Upper Bound', 'Poverty Percent, Age 0-17', '90% CI Lower Bound',
       '90% CI Upper Bound', 'Poverty Estimate, Age 5-17 in Families',
       '90% CI Lower Bound', '90% CI Upper Bound',
       'Poverty Percent, Age 5-17 in Families', '90% CI Lower Bound',
       '90% CI Upper Bound', 'Median Household Income', '90% CI Lower Bound',
       '90% CI Upper Bound', 'Poverty Estimate, Age 0-4', '90% CI Lower Bound',
       '90% CI Upper Bound', 'Poverty Percent, Age 0-4', '90% CI Lower Bound',
       '90% CI Upper Bound']
column_names_dat = ['State FIPS Code', 'Poverty Estimate, All Ages',
       '90% CI Lower Bound', '90% CI Upper Bound', 'Poverty Percent, All Ages',
       '90% CI Lower Bound', '90% CI Upper Bound',
       'Poverty Estimate, Age 0-17', '90% CI Lower Bound',
       '90% CI Upper Bound', 'Poverty Percent, Age 0-17', '90% CI Lower Bound',
       '90% CI Upper Bound', 'Poverty Estimate, Age 5-17 in Families',
       '90% CI Lower Bound', '90% CI Upper Bound',
       'Poverty Percent, Age 5-17 in Families', '90% CI Lower Bound',
       '90% CI Upper Bound', 'Median Household Income', '90% CI Lower Bound',
       '90% CI Upper Bound', 'Poverty Estimate, Age 0-4', '90% CI Lower Bound',
       '90% CI Upper Bound', 'Poverty Percent, Age 0-4', '90% CI Lower Bound',
       '90% CI Upper Bound', 'Name', 'Postal Code']

### Load in Population Data, State Codes, and Crime Key

In [3]:
pop = pd.read_csv('pop_data.csv')
pop['Geographic Area'] = pop['Geographic Area'].apply(lambda x: x.replace('.','').strip())
pop = pd.melt(pop, id_vars=['Geographic Area'], var_name='Year', value_name='Population')
pop['Year'] = pop['Year'].astype(int)

state_codes = pd.read_csv('state_abbr.csv')
abbr = [i for i in state_codes['Code']]

key_to_crimes = pd.read_csv('demo_and_data_key.csv')

### Get population Estimates from Census.gov

In [4]:
def create_pop_df():
    column_names = ['Postal Code', 'Name', 'Poverty Estimate, All Ages',
                           'Poverty Percent, All Ages', 'Poverty Estimate, Age 0-17',
                           'Poverty Percent, Age 0-17', 'Poverty Estimate, Age 5-17 in Families',
                           'Poverty Percent, Age 5-17 in Families', 'Median Household Income',
                           'Poverty Estimate, Age 0-4', 'Poverty Percent, Age 0-4', 'year']
    data = pd.DataFrame(columns=column_names)
    for i in range(1995,2020):
        try:
            download_url = 'https://www2.census.gov/programs-surveys/saipe/datasets/' + str(i) +  '/' + str(i) + '-state-and-county/est' + str(i)[2:] + 'us.xls'
            census = pd.read_excel(download_url, header=None).iloc[4:,:]
            census.columns = column_names_xls
            census = census.iloc[:,[1,2,3,6,9,12,15,18,21,24,27]]
            census.loc[:,'year'] = i
        except:
            try:
                download_url = 'https://www2.census.gov/programs-surveys/saipe/datasets/' + str(i) +  '/' + str(i) + '-state-and-county/est' + str(i)[2:] + 'us.dat'
                census = pd.read_fwf(download_url, header = None).drop([1], axis=1)
                census.columns = column_names_dat
                census = census.iloc[:,[1,4,7,10,13,16,19,22,25,28,29]]
                census.loc[:,'year'] = i
            except:
                pass
        data = data.append(census, sort=True)
        print(f'{i}, {census.shape}\r', end="")
    data = data[column_names]
    return data

In [5]:
data = create_pop_df()
data = data.infer_objects()

### Get Crime Data for each state

In [7]:
def create_crime_data():
    data = pd.DataFrame(columns=['data_year', 'key', 'month_num', 'value'])
    for i, state in enumerate(abbr):
        url = 'https://api.usa.gov/crime/fbi/sapi/api/arrest/states/offense/' + state + '/all/' + str(since) + '/' + str(to) + '?api_key=' + api 
        response = requests.get(url)
        parse = json.loads(response.content)
        df = pd.DataFrame(parse['data'])
        df['Code'] = state
        data = data.append(df,  ignore_index = True, sort = True)
        print(f'{i}\r', end="")
    data.columns = ['Code', 'Year', 'Crime', 'month', 'Incidents']
    data = data[['Year', 'Crime', 'Incidents', 'Code']]
    return data

In [8]:
crime = create_crime_data()

### Get Demographic Data for each Crime

In [145]:
variable = ['aggravated-assault','all-other-offenses','arson','burglary','curfew','disorderly-conduct','dui','drug-grand-total','drug-possesion-marijuana','drug-possesion-opium','drug-possesion-other','drug-possesion-subtotal','drug-possesion-synthetic','drug-sales-marijuana','drug-sales-opium','drug-sales-other','drug-sales-subtotal','drug-sales-synthetic','drunkenness','embezzlement','forgery','fraud','gambling-all-other','gambling-bookmaking','gambling-numbers','gambling-total','human-trafficking-commerical','human-trafficking-servitude','larceny','liqour-laws','motor-vehcile-theft','murder','offense-against-family','prostitution','prostitution-assisting','prostitution-prostitution','prostitution-purchasing','rape','robbery','runaway','sex-offenses','simple-assault','stolen-propery','suspicion','vagrancy','vandalism','weapons',]

def create_crime_demographic_data():
    data = pd.DataFrame(columns=['data_year', 'key', 'month_num', 'value'])
    total = len(variable)*len(abbr)
    print('Total: ', total)
    left = 0
    for i, state in enumerate(abbr):
        for x, crime in enumerate(variable):
            url = 'https://api.usa.gov/crime/fbi/sapi/api/arrest/states/' + state + '/'+ crime +'/race/' + str(since) + '/' + str(to) + '?api_key=' + api 
            response = requests.get(url)
            parse = json.loads(response.content)
            try:
                df = pd.DataFrame(parse['data'])
                df['Code'] = state
                df['Crime'] = crime
                data = data.append(df,  ignore_index = True, sort = True)
            except:
                pass
            left += 1
            print(f'% Done: {int((left/total)*100)}% --- {state} - {i}, {crime} - {x}                       \r', end="")
    data = data.drop(['month_num'], axis = 1)
    data.columns = ['Code', 'Crime', 'Year', 'Race', 'Count']
    data['Crime'] = data['Crime'].apply(lambda x: x.title())
    return data

In [146]:
demographic = create_crime_demographic_data()
demographic.to_csv('demographic_info_by_crime.csv', index=False)

Total:  2397


In [9]:
demographic = pd.read_csv('demographic_info_by_crime.csv')
demographic = demographic.pivot_table(index = ['Year', 'Code', 'Crime'], columns='Race', values='Count', aggfunc=np.sum).reset_index()
demographic['Total'] = demographic['American Indian or Alaska Native'] + demographic['Asian'] + demographic['Black or African American'] + demographic['Native Hawaiian'] + demographic['Unknown'] + demographic['White']

### Total Country Demographics

In [24]:
total_demo = pd.read_csv(
    'state_demographics.txt',
    sep = '\t'
).drop(
    ['Notes','State Code','Race Code', 'Yearly July 1st Estimates Code'],
    axis=1
).dropna()

In [30]:
total_demo.columns = ['State','Year','Race','Population_Totals']
total_demo['Year'] = total_demo['Year'].astype(int)
total_demo['Population_Totals'] = total_demo['Population_Totals'].astype(int)

### Merge Population, Crime Data, and Demographics

In [10]:
crime_and_pop = crime.merge(state_codes, on=['Code']).merge(pop, left_on=['State', 'Year'], right_on = ['Geographic Area', 'Year'])
crime_and_pop = crime_and_pop.drop(['Geographic Area'], axis=1)
crime_and_pop['Incidents'] = crime_and_pop['Incidents'].astype('int')
crime_and_pop['Incident %'] = crime_and_pop['Incidents'] / crime_and_pop['Population']

In [48]:
data = demographic.merge(
    key_to_crimes,
    left_on='Crime',
    right_on='Demographic'
).merge(
    crime_and_pop,
    left_on=['Year', 'Code', 'Data'],
    right_on=['Year', 'Code', 'Crime']
).drop(
    ['Crime_x', 'Crime_y', 'Demographic'],
    axis = 1
)
data = data.rename(columns={"Data": "Crime"})

data = data.melt(
    id_vars=['Year', 'Code', 'Crime', 'Incidents',	'State', 'Abbrev',	'Population',	'Incident %'],
    var_name='Race',
    value_name='Count'
).merge(
    total_demo,
    left_on=['Year', 'State', 'Race'],
    right_on=['Year', 'State', 'Race']
)

### Get total crime rates for each state by race

In [53]:
grouped_data = data[['Year', 'Code', 'Race', 'Count']].groupby(['Year', 'Code','Race']).sum()
grouped_data = grouped_data.reset_index()

### Save to CSV

In [55]:
data.to_csv('combined_data.csv', index=False)